In [109]:
key=""
secret=""

# key api test2


In [110]:
from binance_ft.um_futures import UMFutures
um_futures_client = UMFutures(key=key, secret=secret)
# um_futures_client = UMFutures(key=key, secret=secret, base_url="https://testnet.binancefuture.com")

from binance.spot import Spot
spot_client = Spot(api_key=key, api_secret=secret)

In [111]:

pair = "LPTUSDT"
pair_spot = pair.replace("USDT", "BUSD")

future_info = um_futures_client.exchange_info()['symbols']
kk = [i['filters'] for i in future_info if i['symbol'] == pair][0][1]
print(kk)

print(spot_client.exchange_info(pair_spot)['symbols'][0]['filters'][1])

bid_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])
ask_spot = float(spot_client.book_ticker(pair_spot)['askPrice'])

mark_price = float(spot_client.ticker_price(pair_spot)['price'])
mark_ft = float(um_futures_client.mark_price(pair)['markPrice'])
bid_spot, mark_price, ask_spot, mark_ft


{'stepSize': '0.1', 'filterType': 'LOT_SIZE', 'maxQty': '1000000', 'minQty': '0.1'}
{'filterType': 'LOT_SIZE', 'minQty': '0.01000000', 'maxQty': '9222440.00000000', 'stepSize': '0.01000000'}


(6.04, 6.04, 6.06, 6.0428039)

In [101]:
# spot truoc future sau:
import time
precision_ft = 3
pair_spot = pair.replace("USDT", "BUSD")
um_futures_client.change_leverage(symbol=pair, leverage=4, recvWindow=6000)


bid_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])
ask_spot = float(spot_client.book_ticker(pair_spot)['askPrice'])
mark_spot = float(spot_client.ticker_price(pair_spot)['price'])
price_spot = mark_spot
print("order spot at", price_spot)

usdt = 21
quantity_spot = round(usdt/price_spot,2)
# if quantity_ft < quantity_spot:
#     quantity_ft += 0.1

open_spot = spot_client.new_order(symbol=pair_spot, side="BUY", type="LIMIT", quantity=quantity_spot, price=price_spot, timeInForce="GTC")

order_id_spot = open_spot["orderId"]
while True:
    res_query_spot = spot_client.get_order(symbol=pair_spot, orderId=order_id_spot, recvWindow=6000)
    if res_query_spot['status'] == "FILLED":
        executedQty_spot = float(res_query_spot['executedQty'])
        usdt_value_spot = float(res_query_spot['cummulativeQuoteQty'])
        price_spot = float(res_query_spot['price'])
        print("spot:", price_spot, usdt_value_spot, "quantity:", executedQty_spot, res_query_spot['type'])
        now_ft_price = float(um_futures_client.mark_price(pair)['markPrice']) 
        now_ft_price = round(now_ft_price, precision_ft)
        # round(usdt_value_spot/now_ft_price, 1)
        quantity_ft = round((usdt)/now_ft_price, 1)
        open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="LIMIT", quantity=quantity_ft, price = now_ft_price, timeInForce="GTC")

        # open_future = um_futures_client.new_order(symbol=pair, side="SELL", type="MARKET", quantity=quantity_ft)
        ft_order_id = open_future["orderId"]
        while True:
            res_query_ft = um_futures_client.query_order(symbol=pair, orderId=ft_order_id, recvWindow=6000)
            if res_query_ft['status'] == "FILLED":
                # avg_spot = float(spot_order['price'])
                price_ft = float(res_query_ft['avgPrice'])
                executedQty_future = float(res_query_ft['executedQty'])
                usdt_value_ft = float(res_query_ft['cumQuote'])
                print("future: ", price_ft, usdt_value_ft, "quantity:", executedQty_future, res_query_ft['type'])
                break
            time.sleep(0.1)
        break
    time.sleep(0.1)

order spot at 7.01
spot: 7.01 21.03 quantity: 3.0 LIMIT
future:  6.965 20.895 quantity: 3.0 LIMIT


In [ ]:
# spot_client.cancel_open_orders(symbol=pair_spot)

In [102]:

fee_open = 0 # use busd
fee_close = 0.00075

fee_ft_open = 0.00036
fee_ft_close = 0.00036

fee_spot = (fee_open + fee_close)*usdt_value_spot
fee_future = (fee_ft_open + fee_ft_close) * usdt_value_ft
print("fee spot", fee_spot)
print("fee_future", fee_future)

fee spot 0.015772500000000002
fee_future 0.015044400000000001


In [ ]:
close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)

In [103]:
while True:
    # curret_ft = float(um_futures_client.mark_price(pair)['markPrice'])
    # curret_spot = float(spot_client.ticker_price(pair_spot)['price'])

    curret_ft = float(um_futures_client.book_ticker(pair)['askPrice'])  # > mark price
    curret_spot = float(spot_client.book_ticker(pair_spot)['bidPrice'])  # < mark price

    pnl_ft = (price_ft - curret_ft)/price_ft * usdt_value_ft
    pnl_spot = (curret_spot-price_spot)/price_spot * usdt_value_spot
    print("pnl ft: " ,pnl_ft, "pnl_spot", pnl_spot)
    if pnl_spot + pnl_ft - fee_spot - fee_future > usdt_value_spot * 0.0005 :
        # close position
        close_future = um_futures_client.new_order(symbol=pair,side="BUY",type="MARKET",quantity=executedQty_future)
        close_spot = spot_client.new_order(symbol=pair_spot,side="SELL",type="MARKET",quantity=executedQty_spot)
        print("*****************************")
        print("curret_ft", curret_ft)
        print("curret_spot", curret_spot)
        time.sleep(1)
        print("take order at:")
        print(um_futures_client.get_account_trades(symbol=pair, orderId=close_future['orderId'])[0]['price'])
        print(float(close_spot['fills'][0]['price']))


        break
    time.sleep(0.5)

pnl ft:  0.011999999999998678 pnl_spot -0.05999999999999873
pnl ft:  0.02099999999999902 pnl_spot -0.0899999999999981
pnl ft:  0.01499999999999968 pnl_spot -0.05999999999999873
pnl ft:  0.01499999999999968 pnl_spot -0.05999999999999873
pnl ft:  -0.009000000000000341 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  -0.012000000000001345 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  0.0 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  -0.005999999999999339 pnl_spot -0.05999999999999873
pnl ft:  0.0 pnl_spot -0.029999999999999364
pnl ft:  -0.003000000000001002 pnl_spot -0.029999999999999364
pnl ft:  0.0 pnl_spot -0.029999999999999364
pnl ft:  -0.0030000000

IndexError: list index out of range

In [107]:
close_spot['fills']

[{'price': '7.06000000',
  'qty': '3.00000000',
  'commission': '0.00007405',
  'commissionAsset': 'BNB',
  'tradeId': 4938510}]

In [108]:
# calculate profit
BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])

outcome_spot = sum([float(i['qty']) * float(i['price']) for i in close_spot['fills']])
res_ft_close = um_futures_client.get_account_trades(symbol=pair, orderId=close_future['orderId'])[0]
res_ft_open = um_futures_client.get_account_trades(symbol=pair, orderId=open_future['orderId'])[0]

assert float(res_ft_close["qty"]) == quantity_ft and float(res_ft_open["qty"]) == quantity_ft
commision_close_spot = sum([float(i['commission']) for i in close_spot['fills']])*BNB_price
commision_close_future = float(res_ft_close['commission'])*BNB_price
commision_open_future = float(res_ft_open['commission'])*BNB_price

pnl_ft = float(res_ft_close['realizedPnl'])
pnl_spot = outcome_spot -  usdt_value_spot
all_profit = pnl_ft + pnl_spot - commision_close_future - commision_close_spot - commision_open_future
all_profit


-0.039101568000001426

In [ ]:
# response = um_futures_client.get_account_trades(symbol=pair,orderId=ft_order_id, recvWindow=6000)
# BNB_price = float(spot_client.ticker_price("BNBUSDT")['price'])
# float(response[0]['commission']) * BNB_price / usdt_value_ft